In [1]:
from digi.xbee.devices import *
import re
import time

In [2]:
device = XBeeDevice("/dev/ttyUSB1", 9600)
device.open()


remote_xbee = RemoteXBeeDevice(device, XBee64BitAddress.from_hex_string("0013A20041814D79"))

In [3]:
xbee_message = device.read_data()

In [4]:
remote_device =xbee_message.remote_device
data = bytes(xbee_message.data)
is_broadcast = xbee_message.is_broadcast
timestamp = time.ctime(xbee_message.timestamp)

In [5]:
data2 = []
for val in data:
    print(val, ' ', end='')

83  80  0  84  0  72  0  72  83  1  255  

In [6]:
data2 = []
for i in range(0,len(data)):
    if i == 2:
        data2.append(data[i])
    elif i == 4 and chr(data[i]) != 'E':
        data2.append(data[i])
    elif i == 6 and chr(data[i]) != 'E':
        data2.append(data[i])
    elif i == 9 and chr(data[i]) != 'E':
        byte12 = data[i]*256
    elif i == 10 and chr(data[i]) != 'E':
        data2.append(data[i]+byte12)
    else:
        data2.append(chr(data[i]))

In [7]:
data2


['S', 'P', 0, 'T', 0, 'H', 0, 'H', 'S', 511]

In [8]:
data = "".join(map(str, data2))

In [9]:
regex = 'SP(?P<PIR>[0-9]+|E)T(?P<DHT11_T>[0-9]+|E)H(?P<DHT11_H>[0-9]+|E)HS(?P<YL_69>[0-9]+|EE)'
match = re.search(regex, data)
data_dict = match.groupdict()
print(match)

<re.Match object; span=(0, 12), match='SP0T0H0HS511'>


In [13]:
data_dict

{'PIR': '0',
 'DHT11_T': '0',
 'DHT11_H': '0',
 'YL_69': '511',
 'nodo': b' ROUTER_2',
 'timestamp': 'Mon Nov 30 15:19:54 2020'}

In [11]:
remote_device = remote_device.get_parameter('NI')

In [12]:
data_dict.update(nodo=bytes(remote_device), timestamp=timestamp)